In [1]:
from sqlanalyzer import column_parser
import re
import json

In [2]:
def get_sub_query(query_list):
    pos_delete, pos_where = [len(query_list)], len(query_list)

    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete.append(i)    
        elif line.startswith('WHERE'):
            pos_where = i

    end_of_query = min(pos_delete) 

    copy_query_list = query_list.copy()
    main = next((s for s in copy_query_list if s.startswith('FROM')), 'end of query')
    main_pos = copy_query_list.index(main)
    main_query = copy_query_list[:main_pos]

    if end_of_query == pos_where and end_of_query < len(query_list):
    # when one-line WHERE is the end of query, ie no more GROUP BY or ORDER BY
        main_query.extend(copy_query_list[pos_where:end_of_query+1])
        del copy_query_list[:main_pos]
        del copy_query_list[end_of_query-main_pos-1:]
        
    elif end_of_query == pos_where + 1:
    # when there's more after one-line WHERE, eg GROUP BY/ORDER BY
        main_query.extend(copy_query_list[pos_where:end_of_query])
        del copy_query_list[:main_pos]
        del copy_query_list[end_of_query-main_pos-1:]

    elif end_of_query > pos_where and end_of_query == len(query_list):
    # when multi-line WHERE, no more GROUP BY or ORDER BY
        main_query.extend(copy_query_list[pos_where:end_of_query])
        del copy_query_list[:main_pos]
        del copy_query_list[-(end_of_query-pos_where):]
    
    elif end_of_query > pos_where and end_of_query < len(query_list):
    # when multi-line WHERE, followed by GROUP BY/ORDER BY
        main_query.extend(copy_query_list[pos_where:end_of_query])
        del copy_query_list[:main_pos]
        del copy_query_list[pos_where-main_pos:end_of_query-main_pos]
        del copy_query_list[-(len(query_list) - end_of_query):]
        
    elif end_of_query < pos_where and pos_where == len(query_list):
    # when no WHERE, but GROUP BY/ORDER BY/LIMIT
        del copy_query_list[:main_pos]
        del copy_query_list[end_of_query-1:]
      
    elif end_of_query == pos_where and end_of_query == len(query_list):
    # when no WHERE, no GROUP BY/ORDER BY/LIMIT
        del copy_query_list[:main_pos]
    
    return main_query, copy_query_list


## Type A: one line WHERE, no GROUP BY/ORDER BY/LIMIT
## Type B: one line WHERE, followed by GROUP BY/ORDER BY/LIMIT
## Type C: multi-line WHERE, no GROUP BY/ORDER BY/LIMIT
## Type D: multi-line WHERE, followed by GROUP BY/ORDER BY/LIMIT
## Type E: no WHERE, but GROUP BY/ORDER BY/LIMIT
## Type F: no WHERE, no GROUP BY/ORDER BY/LIMIT

## debug

In [6]:
query = """SELECT sfdc_accounts.platform,\n       sfdc_accounts.mobile_os,\n       sfdc_accounts.service_metadata,\n       sfdc_cases.account,\n       sfdc_cases.num_requests,\n       sfdc_cases.owner,\n       sfdc_accounts.user_id\nFROM sfdc.accounts sfdc_accounts\nLEFT JOIN\n  (SELECT MAX(dt)\n   FROM\n     (SELECT dt\n      FROM sfdc.oppty\n      LEFT JOIN\n        (SELECT MAX(dt)\n         FROM\n           (SELECT DISTINCT dt\n            FROM sfdc.owner AS sfdc_owner) AS dt_owner ON sfdc_oppty.dt = sfdc_cases.dt)\n      LEFT JOIN\n        (SELECT dt\n         FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt)) AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt\nLEFT JOIN sfdc.cases AS sfdc_cases ON sfdc_cases.id = sfdc_accounts.case_id\nWHERE sfdc_cases_oppty.dt > '2020-04-03'\n  AND sfdc_cases_oppty.dt < '2020-05-04'\nORDER BY 1\nGROUP BY 3\nLIMIT 20
"""

In [7]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')


In [11]:
# get_sub_query(query_list)
for i,l in enumerate(query_list): print(i,l)

0 SELECT sfdc_accounts.platform,
1        sfdc_accounts.mobile_os,
2        sfdc_accounts.service_metadata,
3        sfdc_cases.account,
4        sfdc_cases.num_requests,
5        sfdc_cases.owner,
6        sfdc_accounts.user_id
7 FROM sfdc.accounts sfdc_accounts
8 LEFT JOIN
9   (SELECT MAX(dt)
10    FROM
11      (SELECT dt
12       FROM sfdc.oppty
13       LEFT JOIN
14         (SELECT MAX(dt)
15          FROM
16            (SELECT DISTINCT dt
17             FROM sfdc.owner AS sfdc_owner) AS dt_owner ON sfdc_oppty.dt = sfdc_cases.dt)
18       LEFT JOIN
19         (SELECT dt
20          FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt)) AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt
21 LEFT JOIN sfdc.cases AS sfdc_cases ON sfdc_cases.id = sfdc_accounts.case_id
22 WHERE sfdc_cases_oppty.dt > '2020-04-03'
23   AND sfdc_cases_oppty.dt < '2020-05-04'
24 ORDER BY 1
25 GROUP BY 3
26 LIMIT 20


In [9]:
pos_delete, pos_where = [len(query_list)], len(query_list)

for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete.append(i)    
    elif line.startswith('WHERE'):
        pos_where = i

end_of_query = min(pos_delete) 

copy_query_list = query_list.copy()
main = next((s for s in copy_query_list if s.startswith('FROM')), 'end of query')
main_pos = copy_query_list.index(main)
main_query = copy_query_list[:main_pos]


In [10]:
pos_where, end_of_query

(22, 24)

In [14]:
main_query

['SELECT sfdc_accounts.platform,',
 '       sfdc_accounts.mobile_os,',
 '       sfdc_accounts.service_metadata,',
 '       sfdc_cases.account,',
 '       sfdc_cases.num_requests,',
 '       sfdc_cases.owner,',
 '       sfdc_accounts.user_id']

In [13]:
end_of_query > pos_where and end_of_query < len(query_list):


D


In [15]:
# when multi-line WHERE, followed by GROUP BY/ORDER BY
main_query.extend(copy_query_list[pos_where:end_of_query])


In [26]:
main_query, copy_query_list

(['SELECT sfdc_accounts.platform,',
  '       sfdc_accounts.mobile_os,',
  '       sfdc_accounts.service_metadata,',
  '       sfdc_cases.account,',
  '       sfdc_cases.num_requests,',
  '       sfdc_cases.owner,',
  '       sfdc_accounts.user_id',
  "WHERE sfdc_cases_oppty.dt > '2020-04-03'",
  "  AND sfdc_cases_oppty.dt < '2020-05-04'"],
 ['FROM sfdc.accounts sfdc_accounts',
  'LEFT JOIN',
  '  (SELECT MAX(dt)',
  '   FROM',
  '     (SELECT dt',
  '      FROM sfdc.oppty',
  '      LEFT JOIN',
  '        (SELECT MAX(dt)',
  '         FROM',
  '           (SELECT DISTINCT dt',
  '            FROM sfdc.owner AS sfdc_owner) AS dt_owner ON sfdc_oppty.dt = sfdc_cases.dt)',
  '      LEFT JOIN',
  '        (SELECT dt',
  '         FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt)) AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt',
  'LEFT JOIN sfdc.cases AS sfdc_cases ON sfdc_cases.id = sfdc_accounts.case_id'])

In [25]:
# del copy_query_list[:main_pos]
# del copy_query_list[pos_where-main_pos:end_of_query-main_pos]
# del copy_query_list[-(len(query_list) - end_of_query):]

## Type A: one line WHERE, no GROUP BY/ORDER BY/LIMIT
end_of_query == pos_where

In [459]:
query = """SELECT *,
b.ad, c.dfts
   FROM
     (SELECT a.*,
             b.*,
             c.*,
             d.*
      FROM
        (SELECT DISTINCT anonymous_id,
                         user_id
         FROM mapbox_customer_data.segment_identifies
         WHERE dt >= '2018-07-01'
           AND anonymous_id IS NOT NULL
           AND user_id IS NOT NULL ) a
        )
   WHERE context_campaign_name IS NOT NULL 
"""

In [460]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')


In [461]:
get_sub_query(query_list)

(['SELECT *,',
  '       b.ad,',
  '       c.dfts',
  'WHERE context_campaign_name IS NOT NULL'],
 ['FROM',
  '  (SELECT a.*,',
  '          b.*,',
  '          c.*,',
  '          d.*',
  '   FROM',
  '     (SELECT DISTINCT anonymous_id,',
  '                      user_id',
  '      FROM mapbox_customer_data.segment_identifies',
  "      WHERE dt >= '2018-07-01'",
  '        AND anonymous_id IS NOT NULL',
  '        AND user_id IS NOT NULL ) a)'])

In [451]:
pos_delete, pos_where = [len(query_list)], len(query_list)

for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete.append(i)    
    elif line.startswith('WHERE'):
        pos_where = i

end_of_query = min(pos_delete) 

copy_query_list = query_list.copy()
main = next((s for s in copy_query_list if s.startswith('FROM')), 'end of query')
main_pos = copy_query_list.index(main)
main_query = copy_query_list[:main_pos]


In [452]:
pos_where, end_of_query

(15, 16)

In [453]:
main_query

['SELECT *,', '       b.ad,', '       c.dfts']

In [454]:
end_of_query == pos_where + 1

True

In [455]:
main_query.extend(copy_query_list[pos_where:end_of_query+1])
del copy_query_list[:main_pos]
del copy_query_list[end_of_query-main_pos-1:]

In [456]:
# del copy_query_list[end_of_query-1:]

In [457]:
main_query, copy_query_list

(['SELECT *,',
  '       b.ad,',
  '       c.dfts',
  'WHERE context_campaign_name IS NOT NULL'],
 ['FROM',
  '  (SELECT a.*,',
  '          b.*,',
  '          c.*,',
  '          d.*',
  '   FROM',
  '     (SELECT DISTINCT anonymous_id,',
  '                      user_id',
  '      FROM mapbox_customer_data.segment_identifies',
  "      WHERE dt >= '2018-07-01'",
  '        AND anonymous_id IS NOT NULL',
  '        AND user_id IS NOT NULL ) a)'])

## Type B: one line WHERE, followed by GROUP BY/ORDER BY/LIMIT

In [472]:
query = """SELECT *, c.dfsb, c.sdf, a.cbd
   FROM
     (SELECT a.*,
             b.*,
             c.*,
             d.*
      FROM
        (SELECT DISTINCT anonymous_id,
                         user_id
         FROM mapbox_customer_data.segment_identifies
         WHERE dt >= '2018-07-01'
           AND anonymous_id IS NOT NULL
           AND user_id IS NOT NULL ) a
        )
   WHERE context_campaign_name IS NOT NULL 
   GROUP BY 1
   ORDER BY 3
 
"""

In [473]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')
# print(formatted_query)


In [474]:
get_sub_query(query_list)

(['SELECT *,',
  '       c.dfsb,',
  '       c.sdf,',
  '       a.cbd',
  'WHERE context_campaign_name IS NOT NULL'],
 ['FROM',
  '  (SELECT a.*,',
  '          b.*,',
  '          c.*,',
  '          d.*',
  '   FROM',
  '     (SELECT DISTINCT anonymous_id,',
  '                      user_id',
  '      FROM mapbox_customer_data.segment_identifies',
  "      WHERE dt >= '2018-07-01'",
  '        AND anonymous_id IS NOT NULL',
  '        AND user_id IS NOT NULL ) a)'])

In [465]:
pos_delete, pos_where = [len(query_list)], len(query_list)

for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete.append(i)    
    elif line.startswith('WHERE'):
        pos_where = i

end_of_query = min(pos_delete) 

copy_query_list = query_list.copy()
main = next((s for s in copy_query_list if s.startswith('FROM')), 'end of query')
main_pos = copy_query_list.index(main)
main_query = copy_query_list[:main_pos]


In [466]:
pos_where, end_of_query

(16, 17)

In [467]:
end_of_query == pos_where + 1

True

In [468]:
copy_query_list[pos_where:end_of_query]

['WHERE context_campaign_name IS NOT NULL']

In [469]:
main_query.extend(copy_query_list[pos_where:end_of_query])
del copy_query_list[:main_pos]
del copy_query_list[end_of_query-main_pos-1:]

In [470]:
main_query, copy_query_list

(['SELECT *,',
  '       c.dfsb,',
  '       c.sdf,',
  '       a.cbd',
  'WHERE context_campaign_name IS NOT NULL'],
 ['FROM',
  '  (SELECT a.*,',
  '          b.*,',
  '          c.*,',
  '          d.*',
  '   FROM',
  '     (SELECT DISTINCT anonymous_id,',
  '                      user_id',
  '      FROM mapbox_customer_data.segment_identifies',
  "      WHERE dt >= '2018-07-01'",
  '        AND anonymous_id IS NOT NULL',
  '        AND user_id IS NOT NULL ) a)'])

## type C: multi-line WHERE, no GROUP BY/ORDER BY

In [475]:
query = """SELECT email,
       created_date,
       last_mql_date_c,
       mql_flag
FROM
  (SELECT email,
          min(created_date) created_date,
          max(last_mql_date_c) last_mql_date_c,
          CASE
              WHEN max(last_mql_date_c) IS NOT NULL THEN 1
              ELSE 0
          END AS mql_flag,
          sum(CASE
                  WHEN is_deleted = TRUE THEN 1
                  ELSE 0
              END) AS is_deleted
   FROM sales.salesforce_leads
   WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY
   GROUP BY 1)
WHERE mql_flag = 1
  AND is_deleted = 0
  AND dt = '2020'
"""

In [476]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')

In [477]:
get_sub_query(query_list)

(['SELECT email,',
  '       created_date,',
  '       last_mql_date_c,',
  '       mql_flag',
  'WHERE mql_flag = 1',
  '  AND is_deleted = 0',
  "  AND dt = '2020'"],
 ['FROM',
  '  (SELECT email,',
  '          min(created_date) created_date,',
  '          max(last_mql_date_c) last_mql_date_c,',
  '          CASE',
  '              WHEN max(last_mql_date_c) IS NOT NULL THEN 1',
  '              ELSE 0',
  '          END AS mql_flag,',
  '          sum(CASE',
  '                  WHEN is_deleted = TRUE THEN 1',
  '                  ELSE 0',
  '              END) AS is_deleted',
  '   FROM sales.salesforce_leads',
  "   WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY",
  '   GROUP BY 1)'])

In [478]:
pos_delete, pos_where = [len(query_list)], len(query_list)

for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete.append(i)    
    elif line.startswith('WHERE'):
        pos_where = i

end_of_query = min(pos_delete) 

copy_query_list = query_list.copy()
main = next((s for s in copy_query_list if s.startswith('FROM')), 'end of query')
main_pos = copy_query_list.index(main)
main_query = copy_query_list[:main_pos]


In [479]:
pos_where, end_of_query

(19, 22)

In [480]:
end_of_query > pos_where and end_of_query == len(query_list)

True

In [481]:
main_query.extend(copy_query_list[pos_where:end_of_query])
del copy_query_list[:main_pos]
del copy_query_list[-(end_of_query-pos_where):]

In [482]:
main_query, copy_query_list

(['SELECT email,',
  '       created_date,',
  '       last_mql_date_c,',
  '       mql_flag',
  'WHERE mql_flag = 1',
  '  AND is_deleted = 0',
  "  AND dt = '2020'"],
 ['FROM',
  '  (SELECT email,',
  '          min(created_date) created_date,',
  '          max(last_mql_date_c) last_mql_date_c,',
  '          CASE',
  '              WHEN max(last_mql_date_c) IS NOT NULL THEN 1',
  '              ELSE 0',
  '          END AS mql_flag,',
  '          sum(CASE',
  '                  WHEN is_deleted = TRUE THEN 1',
  '                  ELSE 0',
  '              END) AS is_deleted',
  '   FROM sales.salesforce_leads',
  "   WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY",
  '   GROUP BY 1)'])

## Type D: multi-line WHERE, followed by GROUP BY/ORDER BY/LIMIT

In [483]:
query = """SELECT *,
b.dt,
c.abc
   FROM
     (SELECT a.*,
             b.*,
             c.*,
             d.*
      FROM
        (SELECT DISTINCT anonymous_id,
                         user_id
         FROM mapbox_customer_data.segment_identifies
         WHERE dt >= '2018-07-01'
           AND anonymous_id IS NOT NULL
           AND user_id IS NOT NULL ) a
        )
WHERE mql_flag = 1
  AND is_deleted = 0
   GROUP BY 1
   ORDER BY 3
   LIMIT 5
"""

In [484]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')
# print(formatted_query)


In [485]:
get_sub_query(query_list)

(['SELECT *,',
  '       b.dt,',
  '       c.abc',
  'WHERE mql_flag = 1',
  '  AND is_deleted = 0'],
 ['FROM',
  '  (SELECT a.*,',
  '          b.*,',
  '          c.*,',
  '          d.*',
  '   FROM',
  '     (SELECT DISTINCT anonymous_id,',
  '                      user_id',
  '      FROM mapbox_customer_data.segment_identifies',
  "      WHERE dt >= '2018-07-01'"])

In [409]:
pos_delete, pos_where = [len(query_list)], len(query_list)

for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete.append(i)    
    elif line.startswith('WHERE'):
        pos_where = i

end_of_query = min(pos_delete) 

copy_query_list = query_list.copy()
main = next((s for s in copy_query_list if s.startswith('FROM')), 'end of query')
main_pos = copy_query_list.index(main)
main_query = copy_query_list[:main_pos]


In [410]:
pos_where, end_of_query

(15, 17)

In [411]:
end_of_query > pos_where and end_of_query < len(query_list)

True

In [412]:
main_query.extend(copy_query_list[pos_where:end_of_query])
del copy_query_list[:main_pos]
del copy_query_list[pos_where-main_pos:end_of_query-main_pos]
del copy_query_list[-(end_of_query - (pos_where-main_pos)):]

In [413]:
main_query, copy_query_list

(['SELECT *,',
  '       b.dt,',
  '       c.abc',
  'WHERE mql_flag = 1',
  '  AND is_deleted = 0'],
 ['FROM',
  '  (SELECT a.*,',
  '          b.*,',
  '          c.*,',
  '          d.*',
  '   FROM',
  '     (SELECT DISTINCT anonymous_id,',
  '                      user_id',
  '      FROM mapbox_customer_data.segment_identifies',
  "      WHERE dt >= '2018-07-01'"])

## Type E: no WHERE, but GROUP BY/ORDER BY/LIMIT

In [489]:
query = """SELECT *, a.cbdf,f.dbfs
   FROM
     (SELECT a.*,
             b.*,
             c.*,
             d.*
      FROM
        (SELECT DISTINCT anonymous_id,
                         user_id
         FROM mapbox_customer_data.segment_identifies
         WHERE dt >= '2018-07-01'
           AND anonymous_id IS NOT NULL
           AND user_id IS NOT NULL ) a
        )
   GROUP BY 1
   ORDER BY 3
   LIMIT 5
"""

In [490]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')
# print(formatted_query)


In [491]:
get_sub_query(query_list)

(['SELECT *,', '       a.cbdf,', '       f.dbfs'],
 ['FROM',
  '  (SELECT a.*,',
  '          b.*,',
  '          c.*,',
  '          d.*',
  '   FROM',
  '     (SELECT DISTINCT anonymous_id,',
  '                      user_id',
  '      FROM mapbox_customer_data.segment_identifies',
  "      WHERE dt >= '2018-07-01'",
  '        AND anonymous_id IS NOT NULL',
  '        AND user_id IS NOT NULL ) a)',
  'GROUP BY 1',
  'ORDER BY 3'])

In [429]:
pos_delete, pos_where = [len(query_list)], len(query_list)

for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete.append(i)    
    elif line.startswith('WHERE'):
        pos_where = i

end_of_query = min(pos_delete) 

copy_query_list = query_list.copy()
main = next((s for s in copy_query_list if s.startswith('FROM')), 'end of query')
main_pos = copy_query_list.index(main)
main_query = copy_query_list[:main_pos]


In [430]:
for i,l in enumerate(query_list): print(i,l)

0 SELECT *
1 FROM
2   (SELECT a.*,
3           b.*,
4           c.*,
5           d.*
6    FROM
7      (SELECT DISTINCT anonymous_id,
8                       user_id
9       FROM mapbox_customer_data.segment_identifies
10       WHERE dt >= '2018-07-01'
11         AND anonymous_id IS NOT NULL
12         AND user_id IS NOT NULL ) a)
13 GROUP BY 1
14 ORDER BY 3
15 LIMIT 5


In [431]:
pos_where, end_of_query 

(16, 13)

In [432]:
end_of_query < pos_where and pos_where == len(query_list) 

True

In [433]:
del copy_query_list[:main_pos]
del copy_query_list[end_of_query-1:]

In [434]:
main_query, copy_query_list

(['SELECT *'],
 ['FROM',
  '  (SELECT a.*,',
  '          b.*,',
  '          c.*,',
  '          d.*',
  '   FROM',
  '     (SELECT DISTINCT anonymous_id,',
  '                      user_id',
  '      FROM mapbox_customer_data.segment_identifies',
  "      WHERE dt >= '2018-07-01'",
  '        AND anonymous_id IS NOT NULL',
  '        AND user_id IS NOT NULL ) a)'])

## Type F: no WHERE, no GROUP BY/ORDER BY/LIMIT

In [494]:
query = """SELECT *, a.cn, dnf.cvs
   FROM
     (SELECT a.*,
             b.*,
             c.*,
             d.*
      FROM
        (SELECT DISTINCT anonymous_id,
                         user_id
         FROM mapbox_customer_data.segment_identifies
         WHERE dt >= '2018-07-01'
           AND anonymous_id IS NOT NULL
           AND user_id IS NOT NULL ) a
        )
"""

In [495]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')
# print(formatted_query)


In [496]:
get_sub_query(query_list)

(['SELECT *,', '       a.cn,', '       dnf.cvs'],
 ['FROM',
  '  (SELECT a.*,',
  '          b.*,',
  '          c.*,',
  '          d.*',
  '   FROM',
  '     (SELECT DISTINCT anonymous_id,',
  '                      user_id',
  '      FROM mapbox_customer_data.segment_identifies',
  "      WHERE dt >= '2018-07-01'",
  '        AND anonymous_id IS NOT NULL',
  '        AND user_id IS NOT NULL ) a)'])

In [437]:
pos_delete, pos_where = [len(query_list)], len(query_list)

for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete.append(i)    
    elif line.startswith('WHERE'):
        pos_where = i

end_of_query = min(pos_delete) 

copy_query_list = query_list.copy()
main = next((s for s in copy_query_list if s.startswith('FROM')), 'end of query')
main_pos = copy_query_list.index(main)
main_query = copy_query_list[:main_pos]


In [438]:
pos_where, end_of_query 

(13, 13)

In [441]:
c

True

In [443]:
del copy_query_list[:main_pos]

In [444]:
main_query, copy_query_list

(['SELECT *'],
 ['FROM',
  '  (SELECT a.*,',
  '          b.*,',
  '          c.*,',
  '          d.*',
  '   FROM',
  '     (SELECT DISTINCT anonymous_id,',
  '                      user_id',
  '      FROM mapbox_customer_data.segment_identifies',
  "      WHERE dt >= '2018-07-01'",
  '        AND anonymous_id IS NOT NULL',
  '        AND user_id IS NOT NULL ) a)'])